In [1]:
%load_ext autoreload
%autoreload 2

# Settings

In [2]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# only on colab

In [5]:
#!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [6]:
#!tar -xvf data/.pokemon/pokemon_combined.tar --directory data/.pokemon/

# Dataset loading

In [7]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

Using TensorFlow backend.


In [8]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [9]:
from transformation.lmdb_transformer import LmdbTransformer

In [10]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

  DEBUG    | Looking for images in '_training'
  DEBUG    | Looking for images in '_training'


  WARNING  | No files found
  WARNING  | No files found
C:\Users\Khalid\Documents\projects\Generative_Models\transformation\file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


  DEBUG    | Looking for images in '_validation'
  DEBUG    | Looking for images in '_validation'


  WARNING  | No files found
  WARNING  | No files found


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [11]:
_instance_scale=1.0
for data in val_generator:
    print(data['images'].numpy().max())
    break
    #print(data.keys())

1.0


In [12]:
data['images'][0].shape

TensorShape([100, 100, 3])

In [13]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [14]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [15]:
latents_dim

100

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),

            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AE_Dense_reconst_ssim_inverse'
experiments_dir='experiments'+sep_local+model_name

In [18]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [19]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [20]:
from utils.data_and_files.file_utils import create_if_not_exist

In [21]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [22]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonAE_Dense_reconst_ssim_inverse\\var_save_dir'

In [23]:
#to restore trained model, set filepath=_restore

In [24]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1666)              168266    
_________________________________________________________________
dense_5 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_6 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_7 (Dense)              (None, 30000)             50010000  
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0

  WARNING  | None
  WARNING  | None


In [25]:
from evaluation.quantitive_metrics.structural_similarity import prepare_ssim_multiscale

#from statistical.losses_utilities import similarty_to_distance
from statistical.ae_losses import expected_loglikelihood_with_lower_bound as ellwlb

In [26]:
def similarity_to_distance(similarity_fn):
    distance_fn = lambda x, y: 0.5*(1-similarity_fn(x,y))
    return distance_fn

ssim = similarity_to_distance(prepare_ssim_multiscale([ae.batch_size]+ae.get_inputs_shape()))

In [27]:
#ae.compile(metrics=None)
#ae.compile()
ae.compile(loss={'x_logits': lambda x_true, x_logits: ssim(x_true, x_logits)})

Model: "pokemonAE_Dense_reconst_ssim_inverse"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 100)               50003250  
_________________________________________________________________
generative (Functional)      (None, 100, 100, 3)       55742822  
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 105,746,072
Trainable params: 105,745,866
Non-trainable params: 206
_________________________________________________________________
None


In [28]:
ae.output_shape

(None, 100, 100, 3)

# Callbacks

In [29]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-15, 
    patience=10, 
    verbose=1, 
    restore_best_weights=False
)

In [31]:
ms = ModelSaver(filepath=_restore)

In [32]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonAE_Dense_reconst_ssim_inverse\\csv_dir\\pokemonAE_Dense_reconst_ssim_inverse.csv'

In [33]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [36]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().

Epoch 00001: loss improved from inf to -2.18843, saving model to experiments\pokemonAE_Dense_reconst_ssim_inverse\var_save_dir
5000/5000 - 1574s - loss: -2.1884e+00 - psnr: 11.0665 - total_variation: 95196.5547 - ssim_multiscale: 5.6650 - sharpdiff: 5.2703 - mean_absolute_error: 0.2238 - mean_squared_error: 0.0838 - val_loss: -3.5364e+00 - val_psnr: 17.3230 - val_total_variation: 42071.4258 - val_ssim_multiscale: 8.5737 - val_sharpdiff: 10.4573 - val_mean_absolute_error: 0.1072 - val_mean_squared_error: 0.0194
Epoch 2/1000000

Epoch 00002: loss improved from -2.18843 to -2.68267, saving model to experiments\pokemonAE_Dense_reconst_ssim_inverse\var_save_dir
5000/5000 - 1562s - loss: -2.6827e+00 - psnr: 12.3160 - total_variation: 97148.0156 - ssim_multiscale: 6.7037 - sharpdiff: 5.3657 - mean_absolute_error: 0.1793 - mean_squared_error: 0.0598 - val_loss:

5000/5000 - 1531s - loss: -2.7975e+00 - psnr: 12.5889 - total_variation: 92818.5781 - ssim_multiscale: 6.9151 - sharpdiff: 5.6143 - mean_absolute_error: 0.1708 - mean_squared_error: 0.0561 - val_loss: -3.0288e+00 - val_psnr: 13.8637 - val_total_variation: 15999.5869 - val_ssim_multiscale: 7.1838 - val_sharpdiff: 11.7456 - val_mean_absolute_error: 0.1907 - val_mean_squared_error: 0.0422
Epoch 00015: early stopping


# Model Evaluation

## inception_score

In [37]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [38]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=100)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

calculating the inception_score mean ...


100it [52:49, 31.39s/it]

calculating the inception_score sigma ...


C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
1it [00:31, 31.10s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
2it [01:02, 31.09s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
3it [01:33, 31.09s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
4it [02:04, 31.11s/it]C:\ProgramData\Anac

41it [21:42, 32.08s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
42it [22:14, 32.04s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
43it [22:45, 31.84s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
44it [23:17, 31.95s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
45it [23:50, 32.03s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
46it [24:2

83it [43:51, 31.04s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
84it [44:22, 31.04s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
85it [44:53, 31.03s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
86it [45:24, 31.02s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
87it [45:55, 31.02s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\core.py:3923: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
88it [46:2

inception_score mean: dask.array<mean_agg-aggregate, shape=(1,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>, sigma: dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>


## Frechet_inception_distance

In [39]:
is_mean.compute()

array([9.25042361e+165])

In [40]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [41]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'frechet inception distance: {fis_score}')

NameError: name 'training_generator' is not defined

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

In [ ]:
def generate():
    for data in train_generator:
        yield data

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)